# LangGraph 멀티에이전트 시스템
## Writer, Reviewer, Summarizer 협업 시스템

이 노트북에서는 LangGraph를 사용하여 세 개의 에이전트가 협업하는 시스템을 만들어보겠습니다:
- **Writer**: 주어진 주제에 대한 초안을 작성
- **Reviewer**: 작성된 내용을 검토하고 피드백 제공
- **Summarizer**: 최종 내용을 요약

각 에이전트는 특정 역할을 수행하고, LangGraph의 상태 관리를 통해 협업합니다.

In [24]:

import os
from typing import List, Dict, Any, TypedDict, Annotated

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolExecutor
from langgraph.checkpoint.memory import MemorySaver
import operator
from dotenv import load_dotenv


load_dotenv(override=True, dotenv_path="../.env")

print("필요한 라이브러리가 성공적으로 import되었습니다.")

필요한 라이브러리가 성공적으로 import되었습니다.


In [25]:
# 상태 정의 - 멀티에이전트 시스템의 공유 상태
class AgentState(TypedDict):
    topic: str                    # 작성할 주제
    draft: str                    # Writer가 작성한 초안
    review_feedback: str          # Reviewer의 피드백
    revised_content: str          # 피드백을 반영한 수정된 내용
    summary: str                  # Summarizer가 만든 요약
    messages: Annotated[List[BaseMessage], operator.add]  # 에이전트 간 메시지 히스토리
    next_agent: str              # 다음에 실행할 에이전트
    iteration_count: int         # 반복 횟수 (무한 루프 방지)

print("AgentState 클래스가 정의되었습니다.")

AgentState 클래스가 정의되었습니다.


In [26]:
# Azure OpenAI 클라이언트 초기화
from openai import AzureOpenAI

def initialize_client():
    """Azure OpenAI 클라이언트를 초기화합니다."""
    try:
        client = AzureOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_KEY"),
            api_version="2024-02-15-preview"
        )
        return client
    except Exception as e:
        print(f"Azure OpenAI 클라이언트 초기화 실패: {e}")
        print("환경 변수를 확인하거나 Mock Client를 사용합니다.")
        return None

# LLM 래퍼 클래스 (Azure OpenAI 클라이언트를 langchain 스타일로 사용)
class AzureOpenAIWrapper:
    def __init__(self, client, deployment_name="gpt-4o"):
        self.client = client
        self.deployment_name = deployment_name
    
    def invoke(self, messages):
        try:
            # langchain 메시지 형식을 OpenAI 형식으로 변환
            openai_messages = []
            for msg in messages:
                if hasattr(msg, 'type'):
                    role = msg.type if msg.type in ['system', 'user', 'assistant'] else 'user'
                    content = msg.content
                else:
                    role = 'user'
                    content = str(msg)
                openai_messages.append({"role": role, "content": content})
            
            response = self.client.chat.completions.create(
                model=self.deployment_name,
                messages=openai_messages,
                temperature=0.7,
                max_tokens=1500
            )
            
            return AIMessage(content=response.choices[0].message.content)
        except Exception as e:
            print(f"API 호출 실패: {e}")
            return AIMessage(content=f"API 호출 중 오류 발생: {e}")

# Mock LLM 클래스 (환경 설정이 안 된 경우 테스트용)
class MockLLM:
    def invoke(self, messages):
        if isinstance(messages, list):
            last_message = messages[-1].content if messages else ""
        else:
            last_message = str(messages)
        
        return AIMessage(content=f"Mock response for: {last_message[:100]}...")

# 클라이언트 초기화
client = initialize_client()
if client:
    llm = AzureOpenAIWrapper(client, deployment_name="gpt-4o")  # deployment 이름 수정 필요
    print("Azure OpenAI 클라이언트가 성공적으로 초기화되었습니다.")
else:
    llm = MockLLM()
    print("Mock LLM을 사용합니다.")

Azure OpenAI 클라이언트가 성공적으로 초기화되었습니다.


In [27]:
# Writer 에이전트
def writer_agent(state: AgentState) -> Dict[str, Any]:
    """
    주어진 주제에 대한 초안을 작성하는 에이전트
    """
    print("🖊️ Writer 에이전트가 작업을 시작합니다...")
    
    topic = state["topic"]
    
    # 프롬프트 템플릿
    writer_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 전문적인 글쓰기 전문가입니다. 
        주어진 주제에 대해 잘 구조화되고 정보가 풍부한 글을 작성해주세요.
        글은 다음과 같은 구조를 가져야 합니다:
        1. 서론 - 주제 소개
        2. 본론 - 핵심 내용 (2-3개 섹션)
        3. 결론 - 요약 및 마무리
        
        길이는 약 300-500단어 정도로 작성해주세요."""),
        ("human", f"주제: {topic}")
    ])
    
    # LLM 호출
    messages = writer_prompt.format_messages(topic=topic)
    response = llm.invoke(messages)
    
    draft = response.content if hasattr(response, 'content') else str(response)
    
    print(f"✅ Writer가 초안을 완성했습니다. (길이: {len(draft)} 문자)")
    
    return {
        "draft": draft,
        "messages": [HumanMessage(content=f"Writer: 주제 '{topic}'에 대한 초안을 작성했습니다.")],
        "next_agent": "reviewer"
    }

In [28]:
# Reviewer 에이전트
def reviewer_agent(state: AgentState) -> Dict[str, Any]:
    """
    Writer가 작성한 초안을 검토하고 피드백을 제공하는 에이전트
    """
    print("🔍 Reviewer 에이전트가 검토를 시작합니다...")
    
    draft = state["draft"]
    topic = state["topic"]
    iteration_count = state.get("iteration_count", 0)
    
    # 프롬프트 템플릿
    reviewer_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 경험 많은 편집자이자 검토자입니다.
        제공된 글을 다음 기준으로 검토해주세요:
        1. 내용의 정확성과 완성도
        2. 구조와 논리적 흐름
        3. 문체와 가독성
        4. 문법 및 오탈자
        
        검토 후에는 다음 형식으로 피드백을 제공해주세요:
        - 장점: [좋은 점들]
        - 개선사항: [구체적인 개선 제안]
        - 전체 평가: [1-10점 척도]
        
        8점 이상이면 '승인', 그 이하면 '수정 필요'라고 명시해주세요."""),
        ("human", f"주제: {topic}\n\n검토할 글:\n{draft}")
    ])
    
    # LLM 호출
    messages = reviewer_prompt.format_messages()
    response = llm.invoke(messages)
    
    review_feedback = response.content if hasattr(response, 'content') else str(response)
    
    # 승인/수정 필요 여부 판단
    is_approved = "승인" in review_feedback or iteration_count >= 2  # 최대 2회 반복
    next_agent = "summarizer" if is_approved else "reviser"
    
    print(f"✅ Reviewer가 검토를 완료했습니다. 결과: {'승인' if is_approved else '수정 필요'}")
    
    return {
        "review_feedback": review_feedback,
        "messages": [HumanMessage(content=f"Reviewer: 검토 완료 - {'승인' if is_approved else '수정 필요'}")],
        "next_agent": next_agent,
        "iteration_count": iteration_count + 1
    }

In [29]:
# Reviser 에이전트 (수정 담당)
def reviser_agent(state: AgentState) -> Dict[str, Any]:
    """
    Reviewer의 피드백을 바탕으로 초안을 수정하는 에이전트
    """
    print("✏️ Reviser 에이전트가 수정 작업을 시작합니다...")
    
    draft = state["draft"]
    review_feedback = state["review_feedback"]
    topic = state["topic"]
    
    # 프롬프트 템플릿
    reviser_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 숙련된 편집자입니다.
        제공된 피드백을 바탕으로 원본 글을 개선해주세요.
        피드백의 구체적인 지적사항들을 반영하되, 글의 핵심 내용과 구조는 유지해주세요.
        개선된 글만 출력하고, 별도의 설명은 추가하지 마세요."""),
        ("human", f"주제: {topic}\n\n원본 글:\n{draft}\n\n피드백:\n{review_feedback}")
    ])
    
    # LLM 호출
    messages = reviser_prompt.format_messages()
    response = llm.invoke(messages)
    
    revised_content = response.content if hasattr(response, 'content') else str(response)
    
    print("✅ Reviser가 수정을 완료했습니다.")
    
    return {
        "draft": revised_content,  # 수정된 내용을 다시 draft로 설정
        "revised_content": revised_content,
        "messages": [HumanMessage(content="Reviser: 피드백을 반영하여 글을 수정했습니다.")],
        "next_agent": "reviewer"  # 다시 검토 단계로
    }

In [30]:
# Summarizer 에이전트
def summarizer_agent(state: AgentState) -> Dict[str, Any]:
    """
    최종 승인된 글을 요약하는 에이전트
    """
    print("📝 Summarizer 에이전트가 요약을 시작합니다...")
    
    final_content = state.get("revised_content") or state["draft"]
    topic = state["topic"]
    
    # 프롬프트 템플릿
    summarizer_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 전문적인 요약 전문가입니다.
        제공된 글의 핵심 내용을 간결하고 명확하게 요약해주세요.
        요약은 다음과 같은 형식으로 작성해주세요:
        
        **주요 내용:**
        - [핵심 포인트 1]
        - [핵심 포인트 2]
        - [핵심 포인트 3]
        
        **결론:**
        [한 문장으로 정리한 핵심 메시지]
        
        길이는 100-150단어 정도로 작성해주세요."""),
        ("human", f"주제: {topic}\n\n요약할 글:\n{final_content}")
    ])
    
    # LLM 호출
    messages = summarizer_prompt.format_messages()
    response = llm.invoke(messages)
    
    summary = response.content if hasattr(response, 'content') else str(response)
    
    print("✅ Summarizer가 요약을 완료했습니다.")
    
    return {
        "summary": summary,
        "messages": [HumanMessage(content="Summarizer: 최종 글의 요약을 완성했습니다.")],
        "next_agent": "END"
    }

In [32]:
# 라우팅 함수 - 다음 실행할 에이전트 결정
def route_to_next_agent(state: AgentState) -> str:
    """
    현재 상태를 바탕으로 다음에 실행할 에이전트를 결정
    """
    next_agent = state.get("next_agent", "END")
    print(f"🔄 다음 에이전트: {next_agent}")
    return next_agent

# LangGraph 워크플로우 구성
def create_multi_agent_workflow():
    """
    멀티에이전트 워크플로우를 생성합니다.
    """
    # StateGraph 생성
    workflow = StateGraph(AgentState)
    
    # 노드 추가
    workflow.add_node("writer", writer_agent)
    workflow.add_node("reviewer", reviewer_agent)
    workflow.add_node("reviser", reviser_agent)
    workflow.add_node("summarizer", summarizer_agent)
    
    # 엣지 추가 - 조건부 라우팅
    workflow.add_conditional_edges(
        "writer",
        route_to_next_agent,
        {
            "reviewer": "reviewer",
            "END": END
        }
    )
    
    workflow.add_conditional_edges(
        "reviewer",
        route_to_next_agent,
        {
            "summarizer": "summarizer",
            "reviser": "reviser",
            "END": END
        }
    )
    
    workflow.add_conditional_edges(
        "reviser",
        route_to_next_agent,
        {
            "reviewer": "reviewer",
            "END": END
        }
    )
    
    workflow.add_conditional_edges(
        "summarizer",
        route_to_next_agent,
        {
            "END": END
        }
    )
    
    # 시작점 설정
    workflow.set_entry_point("writer")
    
    # 체크포인터 추가 (상태 저장용) - debug 모드
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory, debug=True)
    
    return app

# 워크플로우 생성
multi_agent_app = create_multi_agent_workflow()
print("🚀 멀티에이전트 워크플로우가 생성되었습니다!")


🚀 멀티에이전트 워크플로우가 생성되었습니다!


In [33]:
# 멀티에이전트 시스템 실행 함수
def run_multi_agent_system(topic: str, thread_id: str = "default"):
    """
    멀티에이전트 시스템을 실행합니다.
    
    Args:
        topic (str): 작성할 주제
        thread_id (str): 대화 스레드 ID (상태 관리용)
    """
    print(f"🎯 주제: {topic}")
    print("=" * 50)
    
    # 초기 상태 설정
    initial_state = {
        "topic": topic,
        "draft": "",
        "review_feedback": "",
        "revised_content": "",
        "summary": "",
        "messages": [],
        "next_agent": "writer",
        "iteration_count": 0
    }
    
    # 설정
    config = {"configurable": {"thread_id": thread_id}}
    
    try:
        # 전체 실행 후 최종 상태 가져오기
        final_state = multi_agent_app.invoke(initial_state, config)
        
        print("\n🎉 워크플로우 완료!")
        
        # 각 단계별 결과 출력
        if final_state.get("draft"):
            print(f"\n📋 DRAFT:")
            print("-" * 30)
            draft_preview = final_state["draft"][:300] + "..." if len(final_state["draft"]) > 300 else final_state["draft"]
            print(draft_preview)
        
        if final_state.get("review_feedback"):
            print(f"\n📋 REVIEW_FEEDBACK:")
            print("-" * 30)
            review_preview = final_state["review_feedback"][:200] + "..." if len(final_state["review_feedback"]) > 200 else final_state["review_feedback"]
            print(review_preview)
            
        if final_state.get("summary"):
            print(f"\n📋 SUMMARY:")
            print("-" * 30)
            print(final_state["summary"])
        
        return final_state
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return None

print("✅ 실행 함수가 정의되었습니다.")

✅ 실행 함수가 정의되었습니다.


## 테스트 실행

이제 멀티에이전트 시스템을 테스트해보겠습니다. 다양한 주제로 시스템을 실행해볼 수 있습니다.

In [34]:
# 테스트 1: 인공지능 주제로 실행
print("🧪 테스트 1: 인공지능과 미래사회")
result1 = run_multi_agent_system("인공지능과 미래사회", "test1")

if result1:
    print("\n🎉 최종 결과:")
    print("=" * 50)
    print(f"주제: {result1.get('topic', 'N/A')}")
    print(f"\n최종 글 길이: {len(result1.get('revised_content') or result1.get('draft', ''))}")
    print(f"반복 횟수: {result1.get('iteration_count', 0)}")
    
    # 최종 요약 출력
    if result1.get('summary'):
        print(f"\n📋 최종 요약:")
        print("-" * 30)
        print(result1['summary'])

🧪 테스트 1: 인공지능과 미래사회
🎯 주제: 인공지능과 미래사회
[0:tasks] Starting step 0 with 1 task:
- __start__ -> {'draft': '',
 'iteration_count': 0,
 'messages': [],
 'next_agent': 'writer',
 'review_feedback': '',
 'revised_content': '',
 'summary': '',
 'topic': '인공지능과 미래사회'}
[0:writes] Finished step 0 with writes to 8 channels:
- topic -> '인공지능과 미래사회'
- draft -> ''
- review_feedback -> ''
- revised_content -> ''
- summary -> ''
- messages -> []
- next_agent -> 'writer'
- iteration_count -> 0
[0:checkpoint] State at the end of step 0:
{'draft': '',
 'iteration_count': 0,
 'messages': [],
 'next_agent': 'writer',
 'review_feedback': '',
 'revised_content': '',
 'summary': '',
 'topic': '인공지능과 미래사회'}
[1:tasks] Starting step 1 with 1 task:
- writer -> {'draft': '',
 'iteration_count': 0,
 'messages': [],
 'next_agent': 'writer',
 'review_feedback': '',
 'revised_content': '',
 'summary': '',
 'topic': '인공지능과 미래사회'}
🖊️ Writer 에이전트가 작업을 시작합니다...
✅ Writer가 초안을 완성했습니다. (길이: 1470 문자)
🔄 다음 에이전트: reviewer
[1:write

In [35]:
# 테스트 2: 다른 주제로 실행
print("\n\n🧪 테스트 2: 지속가능한 개발")
result2 = run_multi_agent_system("지속가능한 개발과 환경보호", "test2")

if result2:
    print("\n🎉 최종 결과:")
    print("=" * 50)
    print(f"주제: {result2.get('topic', 'N/A')}")
    print(f"최종 글 길이: {len(result2.get('revised_content') or result2.get('draft', ''))}")
    print(f"반복 횟수: {result2.get('iteration_count', 0)}")
    
    if result2.get('summary'):
        print(f"\n📋 최종 요약:")
        print("-" * 30)
        print(result2['summary'])



🧪 테스트 2: 지속가능한 개발
🎯 주제: 지속가능한 개발과 환경보호
[0:tasks] Starting step 0 with 1 task:
- __start__ -> {'draft': '',
 'iteration_count': 0,
 'messages': [],
 'next_agent': 'writer',
 'review_feedback': '',
 'revised_content': '',
 'summary': '',
 'topic': '지속가능한 개발과 환경보호'}
[0:writes] Finished step 0 with writes to 8 channels:
- topic -> '지속가능한 개발과 환경보호'
- draft -> ''
- review_feedback -> ''
- revised_content -> ''
- summary -> ''
- messages -> []
- next_agent -> 'writer'
- iteration_count -> 0
[0:checkpoint] State at the end of step 0:
{'draft': '',
 'iteration_count': 0,
 'messages': [],
 'next_agent': 'writer',
 'review_feedback': '',
 'revised_content': '',
 'summary': '',
 'topic': '지속가능한 개발과 환경보호'}
[1:tasks] Starting step 1 with 1 task:
- writer -> {'draft': '',
 'iteration_count': 0,
 'messages': [],
 'next_agent': 'writer',
 'review_feedback': '',
 'revised_content': '',
 'summary': '',
 'topic': '지속가능한 개발과 환경보호'}
🖊️ Writer 에이전트가 작업을 시작합니다...
✅ Writer가 초안을 완성했습니다. (길이: 1426 문자)
🔄 다음 에이전

## 시스템 확장 가능성

이 멀티에이전트 시스템은 다음과 같이 확장할 수 있습니다:

### 1. 추가 에이전트
- **Researcher**: 주제에 대한 추가 정보 수집
- **SEO Optimizer**: SEO 최적화 제안
- **Translator**: 다국어 번역
- **Designer**: 레이아웃 및 스타일 제안

### 2. 고급 기능
- **병렬 처리**: 여러 Writer가 동시에 작업
- **A/B 테스트**: 여러 버전의 글 생성 및 비교
- **실시간 협업**: 여러 사용자가 동시에 참여
- **버전 관리**: 글의 변경 히스토리 추적

### 3. 통합 기능
- **외부 API 연동**: 웹 검색, 이미지 생성 등
- **데이터베이스 연동**: 과거 글들을 참고자료로 활용
- **알림 시스템**: 작업 완료 시 알림 발송

### 4. 성능 최적화
- **캐싱**: 반복적인 요청 결과 저장
- **배치 처리**: 여러 주제를 한번에 처리
- **리소스 관리**: 동시 실행 수 제한

In [ ]:
# 사용자 정의 주제로 테스트해보기
def interactive_test():
    """
    사용자가 직접 주제를 입력하여 테스트할 수 있는 함수
    """
    print("🎮 인터랙티브 테스트")
    print("원하는 주제를 입력하여 멀티에이전트 시스템을 테스트해보세요!")
    print("예시: '블록체인 기술의 미래', '원격근무의 장단점', '메타버스와 교육' 등")
    print()
    
    # 예시 주제들
    sample_topics = [
        "딥러닝의 발전과 사회적 영향",
        "클라우드 컴퓨팅의 장점과 보안 이슈",
        "사물인터넷(IoT)과 스마트홈",
        "가상현실(VR) 기술의 교육 활용",
        "자율주행 자동차의 윤리적 문제"
    ]
    
    print("💡 추천 주제들:")
    for i, topic in enumerate(sample_topics, 1):
        print(f"{i}. {topic}")
    
    return sample_topics

# 추천 주제 표시
sample_topics = interactive_test()

# 원하는 주제로 직접 실행해보세요:
# custom_topic = "여기에 원하는 주제를 입력하세요"
# result = run_multi_agent_system(custom_topic, "custom_test")

print("\n📝 사용법:")
print("1. 위의 추천 주제 중 하나를 선택하거나")
print("2. custom_topic 변수에 원하는 주제를 입력하고")
print("3. run_multi_agent_system(custom_topic, 'custom_test') 실행")
print("\n🚀 이제 멀티에이전트 시스템이 준비되었습니다!")